# Estimating Shapley Values

There is more than one way to estimate Shapley values.
The good news: they all have the same result, the Shapley values (with two exceptions, see box).

The estimation methods differ in:

- how fast they are
- how accurate they are, usually as trade-off with fastness
- for which models they can be used

::: {.callout-note}


It's important to know all the estimation methods, as the best one will be picked automatically, based on your model.
Remember: No matter wich estimation you pick, you'll get the same Shapley values, at least in expectation.
Exceptions: Linear SHAP with feature_pertubation="correlation_dependent" and Tree Explainer with feature_pertubation="tree_path_dependent" deliver different results. More on that later.

:::

But first, let's see why it's necessary anyways to have multiple options.

## There are just too many possible coalitions

But why anyways do we need so many algorithms?
In the [chapter on theory](#theory) we already saw that we can enumerate all coalitions and then just compute the marginal contributions for all possible feature coalitions.
The problem: there are just too many coalitions, especially if we have many features.
If we compute the Shapley values for one feature in the case of a model with just two inputs, we have two coalitions to which we can add the feature: the empty coaltion and the one already containing the other feature.
If we have 3 inputs, it's already 4 coalitions.
If we have 4 inputs, it's already 8 coalitions.
In fact, there are $2^(p-1)$ coalitions to which we can add the feature value to, where $p$ is the number of features in the predictive model.

Let's begin, starting with the "simplest" and most accurate one.

## What else is tricky about explanation

There is another part that has to be estimated or where there have to be taken "shortcuts".
The absence of a feature has to be simulated by replacing the feature with a background sample.
But if we would do that only once, this would make for a poor estimate, because then the coalition would wildly depend on the sampled background dataset.
In fact, it would be even more relevant than adding the feature value in question to the coalition.
So we also have to sample here multiple times and average over it.
And thereby remove the effect of the particular sample.



Alright, let's start with the model-agnostic explainers that work for any model.

## Exact Explainer: computing all the coalitions

The exact explainer (in theory) computes  all $2^{(p-1)}$ possible coalitions for each feature, so $2^p$ in total and sums over them as described in the [theory chapter](#theory).
In addition it also uses all of the background data and not just a sample from it.
That means we have no elements of randomness in the computation.
This can be very expensive of course, depending on the number of features and the size of the background data.
But the good side is that we use all available information and get the most accurate estimation of the Shapley values compared to the other model-agnostic estimation methods.

In the shap implementation, however, there are some practical limitations.
We can use the exact method by the folloing code:


In [ ]:
#| eval: false
explainer = shap.explainers.Exact(model, background)

In practice, shap limits marginal contributions of features being added to coalitions of size 0, 1, p-2, and p-1, which means it covers interactions of max size 2.
As the [documentation says](https://github.com/slundberg/shap/blob/master/shap/explainers/_exact.py) it should be less than 15 features.
Due to this enumeration, the exact explainer can make use of "grey code", an optimization method that don't work for the other shap estimation methods.

Grey code is clever ordering of the coalitions so that the coalitions next to each other only differ in one feature value (team member) and therfore can be directly use to compute marginal contributions.
This is more effective than enumerating all possible coalitions and then adding features to them, as gray code reduces the number of model calls because we more efficiently use computations.

Exact shap values are often not feasible.
One remedy is sampling.

## Sampling Explainer: sampling the coalitions

This is one of the first versions that was proposed by Strumbelj and Kononenko.[@strumbelj2014explaining;@strumbelj2010efficient]
The sampling part has to do with sampling in both dimensions: sampling from the background data and sampling the coalitions.

All possible coalitions (sets) of feature values have to be evaluated with and without the j-th feature to calculate the exact Shapley value.
For more than a few features, the exact solution to this problem becomes problematic as the number of possible coalitions exponentially increases as more features are added.
Strumbelj and Kononenko [@strumbelj2014explaining;@strumbelj2010efficient] proposed an approximation with Monte-Carlo sampling:

$$\hat{\phi}_{j}=\frac{1}{M}\sum_{m=1}^M\left(\hat{f}(x^{m}_{+j})-\hat{f}(x^{m}_{-j})\right)$$

where $\hat{f}(x^{m}_{+j})$ is the prediction for x, but with a random number of feature values replaced by feature values from a random data point z, except for the respective value of feature j.
Monte-Carlo sampling just means sampling from the coalitions.
The x-vector $x^{m}_{-j}$ is almost identical to $x^{m}_{+j}$, but the value $x_j^{m}$ is also taken from the sampled z.
Each of these M new instances is a kind of "Frankenstein's Monster" assembled from two instances.
Note that in the following algorithm, the order of features is not actually changed -- each feature remains at the same vector position when passed to the predict function.
The order is only used as a "trick" here:
By giving the features a new order, we get a random mechanism that helps us put together the "Frankenstein's Monster".
For features that appear left of the feature $x_j$, we take the values from the original observations, and for the features on the right, we take the values from a random instance.

**Approximate Shapley estimation for single feature value**:

- Output: Shapley value for the value of the j-th feature
- Required: Number of iterations M, instance of interest x, feature index j, data matrix X, and machine learning model f
  - For all m = 1,...,M:
    - Draw random instance z from the data matrix X
    - Choose a random permutation o of the feature values
    - Order instance x: $x_o=(x_{(1)},\ldots,x_{(j)},\ldots,x_{(p)})$
    - Order instance z: $z_o=(z_{(1)},\ldots,z_{(j)},\ldots,z_{(p)})$
    - Construct two new instances
        - With j: $x_{+j}=(x_{(1)},\ldots,x_{(j-1)},x_{(j)},z_{(j+1)},\ldots,z_{(p)})$
        - Without j: $x_{-j}=(x_{(1)},\ldots,x_{(j-1)},z_{(j)},z_{(j+1)},\ldots,z_{(p)})$
    - Compute marginal contribution: $\phi_j^{m}=\hat{f}(x_{+j})-\hat{f}(x_{-j})$
- Compute Shapley value as the average: $\phi_j(x)=\frac{1}{M}\sum_{m=1}^M\phi_j^{m}$

First, select an instance of interest x, a feature j and the number of iterations M.
For each iteration, a random instance z is selected from the data and a random order of the features is generated.
Two new instances are created by combining values from the instance of interest x and the sample z.
The instance $x_{+j}$ is the instance of interest, but all values in the order after feature j are replaced by feature values from the sample z.
The instance $x_{-j}$ is the same as $x_{+j}$, but in addition has feature j replaced by the value for feature j from the sample z.
The difference in the prediction from the black box is computed:

$$\phi_j^{m}=\hat{f}(x^m_{+j})-\hat{f}(x^m_{-j})$$

All these differences are averaged and result in:

$$\phi_j(x)=\frac{1}{M}\sum_{m=1}^M\phi_j^{m}$$

Averaging implicitly weighs samples by the probability distribution of X.

The procedure has to be repeated for each of the features to get all Shapley values.

A few notes on the shap implementation of the explainer:

- In the shap packag, the default is 2000 samples.
- You can provide the number of samples with the argument `nsamples`
- Default number of samples is `auto`, meaning 1000 times number of features.
- The sampling explainer only accepts the identity link.

But the sampling explainer is not the smartest way to estimate Shapley values, even if it's an improvement over the exact explainer when we have more data.
Next up: the permutation explainer.

## Permutation Explainer: sampling the coalitions, but smarter

The permutation explainer is different from the sampling explainer.
Instead of sampling random coalitions of features, we sample entire permutations of features.
It samples entire permutations and then iterates through it in both directions.
Let's say we have four features $X_1, X_2, X_3$ and $X_4$.

A random permutation would be: $(X_2, X_3, X_1, X_4)$.
We start at the left and compute the marginal contributions:

- Adding $X_2$ to $\emptyset$
- Adding $X_3$ to $\{X_2\}$
- Adding $X_1$ to $\{X_2, X_3\}$
- Adding $X_4$ to $\{X_2, X_3, X_1\}$
- And the other direction:
- Adding $X_4$ to $\emptyset$
- Adding $X_1$ to $\{X_4\}$
- Adding $X_3$ to $\{X_1, X_4\}$
- Adding $X_2$ to $\{X_3, X_1, X_4\}$

This only changes one feature at a time, just like the trick that was used for the Exact explainer.
This minimizes the model calls as the first term of a marginal contribution becomes the second term of the next.
For example the coalition $\{X_2, X_3\}$ is both used for computing the marginal contribution of $\{X_1\}$ to $\{X_2, X_3\}$ and of $X_3$ to $\{X_2\}$.

But this permutation procedure also has another effect: It ensures that the efficiency axiom is always satisfied.
Not only on average, but exactly.
While the individual Shapley values are still estimates though.
To estimate the Shapley values, more than one such permutation has to be sampled and run forward and backward.
Shapley values are then constructed again by averaging the marginal contributions with their appropriate weights.

Another cool thing about the permutation procedure: Just one permutation is enough to get the exact SHAP values for models with up to second order interaction effects.
That's surprising, at first.
Let's see from the above example how this can be:
Let's see for the case of $X_3$, which appears at the second place in the ordering above.
We compute two marginal contributions for this feature:
Adding it $X_3$ to $\{X_2\}$ and adding it to $\{X_1, X_4\}$.

CONTINUE HERE

WHY IS THIS SUFFICIENT FOR 2-wway interactions???


This type of sampling is also called antithetic sampling and fares quite well compared to other sampling estimators of Shapley values.[@mitchell2022sampling]


In [ ]:
explainer = shap.explainers.Permutation(model, background)

Other notes on implementation in shap:

- By default there are 10 such permutations that are iterated forward and backward, in the current shap implementation.
- allows for hierarchical data structures with partition trees. not implemented in Kernel explainer (see later) or sampling explainer
- Contains random component (sampling) and therefore recommended to set a seed.




## Additive Explainer

- [code](https://github.com/slundberg/shap/blob/master/shap/explainers/_additive.py)
- assumes that model only has first order effects
- for GAMs
- so like permutation explainer, but only uses 1er effects
- how it works:
- by knowing that the model only has first order effects, we don't need to compute all the coaliations
- we know that the effect of adding a feature is the same, no matter which features are already in there
- that means, we can simply compute the p-1 models where we take and add each of the individual features
- only works with Tabular masker



## Linear Explainer

- SHAP for linear models
- $\beta_j \cdot (x_i - \mathbb{E}[f(X)])$
- makes use of the linear structure
- directly works with the coefficients
- works with link function
- because of that can work with genrealized linar model
- logistic regression being a popular example of that
- when feature_perturbation set to "correlation_dependent", computes different shapley values (for different game)
-  TODO: link to relevant chapter
- 

## Kernel Explainer

<!-- The general Idea of linear model -->
KernelSHAP estimates for an instance x the contributions of each feature value to the prediction.
KernelSHAP consists of five steps:

- Sample coalitions $z_k'\in\{0,1\}^M,\quad{}k\in\{1,\ldots,K\}$ (1 = feature present in coalition, 0 = feature absent).
- Get prediction for each $z_k'$ by first converting $z_k'$ to the original feature space and then applying model $\hat{f}: \hat{f}(h_x(z_k'))$
- Compute the weight for each $z_k'$ with the SHAP kernel.
- Fit weighted linear model.
- Return Shapley values $\phi_k$, the coefficients from the linear model.

We can create a random coalition by repeated coin flips until we have a chain of 0's and 1's.
For example, the vector of (1,0,1,0) means that we have a coalition of the first and third features.
The K sampled coalitions become the dataset for the regression model.
The target for the regression model is the prediction for a coalition.
("Hold on!," you say. "The model has not been trained on these binary coalition data and cannot make predictions for them.")
To get from coalitions of feature values to valid data instances, we need a function $h_x(z')=z$ where $h_x:\{0,1\}^M\rightarrow\mathbb{R}^p$.
The function $h_x$ maps 1's to the corresponding value from the instance x that we want to explain.
For tabular data, it maps 0's to the values of another instance that we sample from the data.
This means that we equate "feature value is absent" with "feature value is replaced by random feature value from data".
For tabular data, the following figure visualizes the mapping from coalitions to feature values:

![Function $h_x$ maps a coalition to a valid instance. For present features (1), $h_x$ maps to the feature values of x. For absent features (0), $h_x$ maps to the values of a randomly sampled data instance.](images/shap-simplified-features.jpg)

$h_x$ for tabular data treats feature $X_j$ and $X_{-j}$ (the other features) as independent and integrates over the marginal distribution:

$$\hat{f}(h_x(z'))=E_{X_{-j}}[\hat{f}(x)]$$

Sampling from the marginal distribution means ignoring the dependence structure between present and absent features.
KernelSHAP therefore suffers from the same problem as all permutation-based interpretation methods.
The estimation puts too much weight on unlikely instances.
Results can become unreliable.
But it is necessary to sample from the marginal distribution.
The solution would be to sample from the conditional distribution, which changes the value function, and therefore the game to which Shapley values are the solution.
As a result, the Shapley values have a different interpretation:
For example, a feature that might not have been used by the model at all can have a non-zero Shapley value when the conditional sampling is used.
For the marginal game, this feature value would always get a Shapley value of 0, because otherwise it would violate the Dummy axiom.

For images, the following figure describes a possible mapping function:

![Function $h_x$ maps coalitions of superpixels (sp) to images. Superpixels are groups of pixels. For present features (1), $h_x$ returns the corresponding part of the original image. For absent features (0), $h_x$ greys out the corresponding area. Assigning the average color of surrounding pixels or similar would also be an option.](images/shap-superpixel.jpg)

<!-- Kernel -->
The big difference to LIME is the weighting of the instances in the regression model.
LIME weights the instances according to how close they are to the original instance.
The more 0's in the coalition vector, the smaller the weight in LIME.
SHAP weights the sampled instances according to the weight the coalition would get in the Shapley value estimation.
Small coalitions (few 1's) and large coalitions (i.e. many 1's) get the largest weights.
The intuition behind it is:
We learn most about individual features if we can study their effects in isolation.
If a coalition consists of a single feature, we can learn about this feature's isolated main effect on the prediction.
If a coalition consists of all but one feature, we can learn about this feature's total effect (main effect plus feature interactions).
If a coalition consists of half the features, we learn little about an individual feature's contribution, as there are many possible coalitions with half of the features.
To achieve Shapley compliant weighting, Lundberg et al. propose the SHAP kernel:

$$\pi_{x}(z')=\frac{(M-1)}{\binom{M}{|z'|}|z'|(M-|z'|)}$$

Here, M is the maximum coalition size and $|z'|$ the number of present features in instance z'.
Lundberg and Lee show that linear regression with this kernel weight yields Shapley values.
If you would use the SHAP kernel with LIME on the coalition data, LIME would also estimate Shapley values!

<!-- Sampling trick -->
The sampling trick:
We can be a bit smarter about the sampling of coalitions:
The smallest and largest coalitions take up most of the weight.
We get better Shapley value estimates by using some of the sampling budget K to include these high-weight coalitions instead of sampling blindly.
We start with all possible coalitions with 1 and M-1 features, which makes 2 times M coalitions in total.
When we have enough budget left (current budget is K - 2M), we can include coalitions with 2 features and with M-2 features and so on.
From the remaining coalition sizes, we sample with readjusted weights.



<!-- Linear Model -->
We have the data, the target and the weights;
Everything we need to build our weighted linear regression model:

$$g(z')=\phi_0+\sum_{j=1}^M\phi_jz_j'$$

We train the linear model g by optimizing the following loss function L:

$$L(\hat{f},g,\pi_{x})=\sum_{z'\in{}Z}[\hat{f}(h_x(z'))-g(z')]^2\pi_{x}(z')$$

where Z is the training data.
This is the good old boring sum of squared errors that we usually optimize for linear models.
The estimated coefficients of the model, the $\phi_j$'s, are the Shapley values.


Since we are in a linear regression setting, we can also make use of the standard tools for regression.
For example, we can add regularization terms to make the model sparse.
If we add an L1 penalty to the loss L, we can create sparse explanations.
(I am not so sure whether the resulting coefficients would still be valid Shapley values though.)


## Tree Explainer

- Tree explainer is, in my opinion, the worke horse
- two ingredients:
- popularity of xgboost, other tree boosting and random forest preparse the ground
- Tree explainer is very efficient and fast, compared to other explainers
- these two facts, in combination, make it feasible and popular to use shap with state of the arte machine learning
-

Lundberg et al. (2018)[^treeshap] proposed TreeSHAP, a variant of SHAP for tree-based machine learning models such as decision trees, random forests and gradient boosted trees.
TreeSHAP was introduced as a fast, model-specific alternative to KernelSHAP, but it turned out that it can produce unintuitive feature attributions.

TreeSHAP defines the value function using the conditional expectation $E_{X_j|X_{-j}}(\hat{f}(x)|x_j)$ instead of the marginal expectation.
The problem with the conditional expectation is that features that have no influence on the prediction function f can get a TreeSHAP estimate different from zero as shown by Sundararajan et al. (2019) [^cond1] and Janzing et al. (2019) [^cond2].
The non-zero estimate can happen when the feature is correlated with another feature that actually has an influence on the prediction.

How much faster is TreeSHAP?
Compared to exact KernelSHAP, it reduces the computational complexity from $O(TL2^M)$ to $O(TLD^2)$, where T is the number of trees, L is the maximum number of leaves in any tree and D the maximal depth of any tree.

<!-- To explain an individual prediction with exact Shapley values, we have to estimate  $E(\hat{f}(x)|x_S)$ for all possible feature value subsets S.-->
TreeSHAP uses the conditional expectation $E_{X_j|X_{-j}}(\hat{f}(x)|x_j)$ to estimate effects.
I will give you some intuition on how we can compute the expected prediction for a single tree, an instance x and feature subset S.
If we conditioned on all features -- if S was the set of all features -- then the prediction from the node in which the instance x falls would be the expected prediction.
If we would not condition the prediction on any feature -- if S was empty -- we would use the weighted average of predictions of all terminal nodes.
If S contains some, but not all, features, we ignore predictions of unreachable nodes.
Unreachable means that the decision path that leads to this node contradicts values in $x_S$.
From the remaining terminal nodes, we average the predictions weighted by node sizes (i.e. number of training samples in that node).
The mean of the remaining terminal nodes, weighted by the number of instances per node, is the expected prediction for x given S.
The problem is that we have to apply this procedure for each possible subset S of the feature values.
<!--
This means $\sum_{i=1}{p}\frac{(p-i)!i!}{i!}$ times.
Here, each summand is the set of all possible subsets S with the same cardinality (e.g. all possible subsets with 2 features).
-->
TreeSHAP computes in polynomial time instead of exponential.
The basic idea is to push all possible subsets S down the tree at the same time.
For each decision node we have to keep track of the number of subsets.
This depends on the subsets in the parent node and the split feature.
For example, when the first split in a tree is on feature x3, then all the subsets that contain feature x3 will go to one node (the one where x goes).
Subsets that do not contain feature x3 go to both nodes with reduced weight.
Unfortunately, subsets of different sizes have different weights.
The algorithm has to keep track of the overall weight of the subsets in each node.
This complicates the algorithm.
I refer to the original paper for details of TreeSHAP.
The computation can be expanded to more trees:
Thanks to the Additivity property of Shapley values, the Shapley values of a tree ensemble is the (weighted) average of the Shapley values of the individual trees.


- has regularization parameter
- permutation explainer is better optimized
- kernel explainer has become more or less obsolete and replaced by permutation explainer
- 


## Gradient Explainer

- works for any models with a gradient method
- so gradient-based models
- neural network based models also have a Deep Explainer
- explains model using expected gradients
- can be seen as extension of integrated gradients method
- CITE Sundarajan et al. 2017
- Integrated gradients in contrast have only a single "background" data point.
- more of a reference point really
- but technically also possible in ShAP
- adaption so that shapley values come out:
- integral reformulated as expectation and expectation sampels from entire background, not only the one data point
- works with keras model
- alsow works with PyTorh with nn.Module
-

## Deep Explainer

- enhancement of DeepLIFT algorithm
- similar to Kernel Explainer
- uses background samples
- TODO: cite deep lift
- TODO: CITE original shap paper
-


## Partition Explainer

- based on hierarchical structure of the features
- any tree of features can do
- use case: have correlated features clustered
- computes Shapley values recursively through hierarchy of features
- results in Owen values from game theory
- TODO: decide whether to include or not in book

## When to Use Which

- shap has auto function so you kinda don't have to think about it too much
- when calling "Explainer" default is auto
- don't use kernelshap
- use treeSHAP when you have treeBased model
- use DeepExplainer with neural networks, but only if input should be granular (better partition explainer for images 

## tl;dr / Summary of the chapter

- you don't really have to know
- model-specific used whenever possible
  - tree for tree
  - linear for linear
  - additive for additive
- model-agnostic default is permutation explainer
- kernel explainer only there for "historical" reasons

## What the "auto" option does

- Auto chooses based on the model
- attempts to make the best choice
- is the default if you don't define anything
- can be overriden by specifying the `algorithm` option
